In [53]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
import re
import gender_classification as gc

In [2]:
# load data
df = pd.read_pickle('C:/Users/Ana/OneDrive - Hochschule Düsseldorf/MA/data_topics.pkl')

In [3]:
# add year column
df['year'] = df['date'].dt.year

In [4]:
df.head()

,text,date,legislative_period,speaker_name,speaker_gender,speaker_role,speaker_party,comments,id,speaker_id,speech_length,topic_distribution,year
0,Sehr geehrter Herr Alterspräsident! So muss ...,2021-10-26,20,Gabriele Katzmarek,female,None,SPD,[(Beifall bei der SPD sowie bei Abgeordneten d...,SP-2021-0,1299,2593,{68: 0.35882655},2021
1,Herr Präsident! Liebe Kolleginnen und Kolleg...,2021-10-26,20,Stefan Müller,male,None,CDU/CSU,[(Beifall bei der CDU/CSU sowie bei Abgeordnet...,SP-2021-1,4028,3796,{68: 0.46776655},2021
2,Sehr geehrter Herr Alterspräsident Wolfgang ...,2021-10-26,20,Britta Haßelmann,female,None,BÜNDNIS 90/DIE GRÜNEN,[(Beifall beim BÜNDNIS 90/DIE GRÜNEN und bei d...,SP-2021-2,451,4301,{68: 0.26328164},2021
3,Sehr geehrter Herr Präsident! Meine lieben K...,2021-10-26,20,Marco Buschmann,male,None,FDP,"[(Jan Korte [DIE LINKE]: Oder Jugendweihe!), (...",SP-2021-3,3083,4555,{},2021
4,Herr Alterspräsident! Lassen Sie mich zunäch...,2021-10-26,20,Stephan Brandner,male,None,AfD,"[(Beifall bei der AfD), (Beifall bei Abgeordne...",SP-2021-4,4055,5996,{68: 0.3120498},2021


In [5]:
# filter dataframe for speeches with comments
df_comments = df[df['comments'].apply(lambda x: len(x) > 0)]

In [6]:
df_comments["comments"][1]

['(Beifall bei der CDU/CSU sowie bei Abgeordneten der SPD, des BÜNDNISSES\xa090/DIE GRÜNEN und der FDP)',
 '(Beatrix von Storch [AfD]: Aber die Geschäftsordnung!)',
 '(Beifall bei der CDU/CSU, der SPD, dem BÜNDNIS\xa090/DIE GRÜNEN, der FDP und der LINKEN)',
 '(Zuruf von der AfD: Frechheit!)',
 '(Beifall bei der CDU/CSU, der SPD, dem BÜNDNIS\xa090/DIE GRÜNEN und der LINKEN sowie bei Abgeordneten der FDP)',
 '(Beifall bei der CDU/CSU sowie bei Abgeordneten der SPD und der FDP\xa0– Dr.\xa0Alice Weidel [AfD]: Dafür haben Sie auch die Quittung gekriegt!\xa0– Jan Korte [DIE LINKE]: Das war ganz gut!\xa0– Gegenruf der Abg. Dr.\xa0Alice Weidel [AfD]: Na ja!)',
 '(Beifall beim BÜNDNIS\xa090/DIE GRÜNEN)']

In [101]:
preamble = re.compile(r'(((Weiterer )?Gegenruf (des|der) Abg\.)|(Zuruf (des|der) Abg\.))(?P<name>[\w,\s\.]*)')

def get_speaker_gender(name):
    match = re.search(preamble, name)
    if(match):
        name = match.groupdict()["name"]
    
    name = name.strip()
    if "." in name.split()[0]:
        first_name = name.split()[1]
    else:
        first_name = name.split()[0]
    
    return gc.get_gender(first_name)

In [47]:
# recognizes comments of named speakers
RE_SPEAKER_DIRECTLY = re.compile(r'(?P<speaker>[\w,\s\.]*) \[(?P<party>[\w,/\s]*)\]: (?P<comment>[\w,!\s]*)( - )*')

RES = [RE_SPEAKER_DIRECTLY]

comment_counts = {}

f = open('reference files/comments_per_speech.txt', 'w', encoding='utf-8')

for index, speech in df_comments.iterrows():
    comment_count = 0

    for comment in speech['comments']:
        comment = comment.replace(u'\xa0', u' ')
        for RE in RES:
            matches = RE.finditer(comment)
            for match in matches:
                print(f"{index}: {match.groupdict()}", file=f)
                comment_count += 1

    comment_counts[index] = comment_count

f.close()

In [51]:
# recognizes comments of named speakers
RE_SPEAKER_DIRECTLY = re.compile(r'(?P<speaker>[\w,\s\.]*) \[(?P<party>[\w,/\s]*)\]: (?P<comment>[\w,!\s]*)( - )*')

RES = [RE_SPEAKER_DIRECTLY]

comment_counts = {}
comment_speakers = {}

#f = open('reference files/comments_per_speech.txt', 'w', encoding='utf-8')

for index, speech in df_comments.iterrows():
    comment_count = 0
    speakers = []

    for comment in speech['comments']:
        comment = comment.replace(u'\xa0', u' ')
        for RE in RES:
            matches = RE.finditer(comment)
            for match in matches:
                #print(f"{index}: {match.groupdict()}", file=f)
                comment_count += 1
                speakers.append(match.groupdict()["speaker"].strip())

    comment_counts[index] = comment_count
    comment_speakers[index] = speakers

#f.close()

In [102]:
get_speaker_gender("Gegenruf der Abg. Dr. Alice Weidel")

'female'

In [ ]:
#TODO add columns for comment speaker gender count

In [52]:
comment_speakers

{0: [],
 1: ['Beatrix von Storch',
  'Dr. Alice Weidel',
  'Jan Korte',
  'Gegenruf der Abg. Dr. Alice Weidel'],
 2: ['Stephan Brandner', 'Dr. Alice Weidel', 'Enrico Komning'],
 3: ['Jan Korte'],
 4: ['',
  'Britta Haßelmann',
  'Amira Mohamed Ali',
  '',
  'Ralph Brinkhaus',
  'Dorothee Bär'],
 5: ['Dr. Bernd Baumann',
  'Dr. Bernd Baumann',
  'Norbert Kleinwächter',
  'Christian Lindner',
  'Dr. Marco Buschmann',
  'Dr. Marco Buschmann',
  'Enrico Komning',
  'Alexander Dobrindt',
  'Jan Korte',
  'Jan Korte'],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: ['Brömer', 'Stephan Brandner', 'Jan Korte', 'Andrea Lindholz'],
 15: ['Gabriele Katzmarek',
  'Andrea Lindholz',
  'Andrea Lindholz',
  'Andrea Lindholz',
  'Dr. Marco Buschmann',
  'Dr. Marco Buschmann',
  'Dr. Marco Buschmann',
  'Jan Korte'],
 16: ['Brömer', 'Stephan Brandner', 'Brömer', 'Dr. Bernd Baumann'],
 17: ['Brömer', 'Gabriele Katzmarek', 'Gabriele Katzmarek', 'Yasmin Fahimi'],
 18: ['Brömer'],

In [28]:
# get comment count by speech id
comment_counts[568537]

7

In [ ]:
# add column for comment count
df_comments['comment_count'] = comment_counts.values()

In [31]:
df_comments.head()

,text,date,legislative_period,speaker_name,speaker_gender,speaker_role,speaker_party,comments,id,speaker_id,speech_length,topic_distribution,year,comment_count
0,Sehr geehrter Herr Alterspräsident! So muss ...,2021-10-26,20,Gabriele Katzmarek,female,None,SPD,[(Beifall bei der SPD sowie bei Abgeordneten d...,SP-2021-0,1299,2593,{68: 0.35882655},2021,0
1,Herr Präsident! Liebe Kolleginnen und Kolleg...,2021-10-26,20,Stefan Müller,male,None,CDU/CSU,[(Beifall bei der CDU/CSU sowie bei Abgeordnet...,SP-2021-1,4028,3796,{68: 0.46776655},2021,4
2,Sehr geehrter Herr Alterspräsident Wolfgang ...,2021-10-26,20,Britta Haßelmann,female,None,BÜNDNIS 90/DIE GRÜNEN,[(Beifall beim BÜNDNIS 90/DIE GRÜNEN und bei d...,SP-2021-2,451,4301,{68: 0.26328164},2021,3
3,Sehr geehrter Herr Präsident! Meine lieben K...,2021-10-26,20,Marco Buschmann,male,None,FDP,"[(Jan Korte [DIE LINKE]: Oder Jugendweihe!), (...",SP-2021-3,3083,4555,{},2021,1
4,Herr Alterspräsident! Lassen Sie mich zunäch...,2021-10-26,20,Stephan Brandner,male,None,AfD,"[(Beifall bei der AfD), (Beifall bei Abgeordne...",SP-2021-4,4055,5996,{68: 0.3120498},2021,6


In [13]:
# test comment count per speech
RE_SPEAKER_DIRECTLY = re.compile(r'(?P<speaker>[\w,\s\.]*) \[(?P<party>[\w,/\s]*)\]: (?P<comment>[\w,!\s]*)( - )*') # TODO erweitern?

RES = [RE_SPEAKER_DIRECTLY]

comment_count = 0

for comment in df_comments["comments"][1]:
    comment = comment.replace(u'\xa0', u' ')
    for RE in RES:
        matches = RE.finditer(comment)
        for match in matches:
            print(match.groupdict())
            comment_count += 1

{'speaker': 'Beatrix von Storch', 'party': 'AfD', 'comment': 'Aber die Geschäftsordnung!'}
{'speaker': ' Dr. Alice Weidel', 'party': 'AfD', 'comment': 'Dafür haben Sie auch die Quittung gekriegt! '}
{'speaker': ' Jan Korte', 'party': 'DIE LINKE', 'comment': 'Das war ganz gut! '}
{'speaker': ' Gegenruf der Abg. Dr. Alice Weidel', 'party': 'AfD', 'comment': 'Na ja!'}


In [33]:
# calculate average comment count per speech by gender
average_comments_by_gender = df_comments.groupby('speaker_gender')['comment_count'].mean()
average_comments_by_gender

speaker_gender
female    1.845188
male      1.891020
Name: comment_count, dtype: float64